# Association Rule Mining
- Generate synthetic transactional data (e.g., 20-50 transactions, each a list of 3-8 items from a pool of 20 items like ['milk', 'bread', 'beer', 'diapers', 'eggs', etc.]; use random.choices to create baskets with patterns, like frequent co-occurrences). Use Python lists or
pandas. Include generation code.
2. Analyze: Discuss one rule's implications (e.g., for retail recommendations).

In [15]:
# Import libraries
import pandas as pd
import random
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

# Generate Synthetic transaction data
random.seed(42)
num_transactions = random.randint(20, 50)
item_pool = ['milk', 'bread', 'beer', 'diapers', 'eggs', 'fruit', 'vegetables', 'snacks', 'cereal', 'juice',
            'toilet paper', 'detergent', 'meat', 'fish', 'pasta', 'rice', 'beans', 'soda', 'water', 'coffee']
transactions = []

for _ in range(num_transactions):
    basket_size = random.randint(3, 8)
    basket = random.choices(item_pool, k=basket_size)
    # remove duplicates
    transactions.append(list(set(basket)))

# Convert to DataFrame
df = pd.DataFrame({'Transaction': range(1, num_transactions + 1),
                'Items': transactions})
df.head()

,Transaction,Items
0,1,"[fruit, eggs, milk]"
1,2,"[milk, bread, detergent, beer, eggs, toilet pa..."
2,3,"[milk, beans, detergent, diapers, eggs, toilet..."
3,4,"[vegetables, diapers, coffee, fish]"
4,5,"[fruit, vegetables, beer, snacks]"


Prepare Data for Mining - One-Hot Encoding

In [16]:
# Convert to One-Hot Encoding
all_items = sorted(set(item for sublist in transactions for item in sublist))
encoded_df = pd.DataFrame(0, index=range(len(transactions)), columns=all_items)

for idx, basket in enumerate(transactions):
    for item in basket:
        encoded_df.loc[idx, item] = 1

Apply Apriori Algorithm to find most frequent itemsets

In [17]:
# Apply Apriori
frequent_itemsets = apriori(encoded_df, min_support=0.1, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.175,(beans)
1,0.125,(beer)
2,0.350,(bread)
3,0.125,(cereal)
4,0.175,(coffee)
5,0.200,(detergent)
6,0.300,(diapers)
7,0.350,(eggs)
8,0.250,(fish)
9,0.300,(fruit)


Generating an association rules

In [18]:
# Generating association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values(by="lift", ascending=False)

print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
if not rules.empty:
    best_rule = rules.iloc[0]
    antecedent = ', '.join(list(best_rule['antecedents']))
    consequent = ', '.join(list(best_rule['consequents']))
    print("\nExample Rule Analysis:")
    print(f"If a customer buys [{antecedent}], they are likely to also buy [{consequent}] "
        f"(confidence: {best_rule['confidence']:.2f}, lift: {best_rule['lift']:.2f}).")
    print("Implication: This could be used in retail for product bundling or targeted recommendations.")


Association Rules:
       antecedents     consequents  support  confidence      lift
75         (juice)   (fish, fruit)      0.1    0.400000  4.000000
74   (fish, fruit)         (juice)      0.1    1.000000  4.000000
73  (juice, fruit)          (fish)      0.1    1.000000  4.000000
76          (fish)  (juice, fruit)      0.1    0.400000  4.000000
72   (juice, fish)         (fruit)      0.1    0.800000  2.666667
..             ...             ...      ...         ...       ...
10         (bread)  (toilet paper)      0.1    0.285714  1.038961
13         (bread)         (water)      0.1    0.285714  1.038961
12         (water)         (bread)      0.1    0.363636  1.038961
27       (diapers)          (soda)      0.1    0.333333  1.025641
26          (soda)       (diapers)      0.1    0.307692  1.025641

[78 rows x 5 columns]

Example Rule Analysis:
If a customer buys [juice], they are likely to also buy [fish, fruit] (confidence: 0.40, lift: 4.00).
Implication: This could be used in reta